# Automated Sensing of Red vs. White Wine
_... with the cheapest sensor possible_

#### Data Science 420 BB
#### University of Washington Professional & Continuing Education

#### Homework 2: Low-cost Sensor, Desigion Trees
#### Leo Salemann, 4/25/18

# OBJECTIVES

1. Design a simple, low-cost sensor that can distinguish between red wine and white wine for at least 95%
of the samples. In other words, achive 95% accuracy with the minimum number of features.
1. Assess Accuracy when using all sensors
1. Find the ninimum number of sensors that still maintains 90% accuracy or better.

# LOAD DATA

In [1]:
file <- "./RedWhiteWine.csv"
data <- read.csv(file, header=TRUE, sep=",", stringsAsFactors = FALSE)

# If the target column, say column class, is numerical, you need to convert it to factors in R for classification problem
data$Class <- factor(data$Class)
head(data)
dim(data)


fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,Class
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,1
7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5,1
7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5,1
11.2,0.28,0.56,1.9,0.075,17,60,0.9980,3.16,0.58,9.8,6,1
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,1
7.4,0.66,0.00,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5,1


[1] 6497   13

In [2]:
install.packages("dplyr")
library(dplyr)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



## Redefine Class 1 as Red and 0 as White

# Make a function that quickly tests Decision Tree accuracy

Very similar to the function we had in HW 1, just replacing the nbc part with a decision tree.

In [3]:
# Classification Tree with rpart
install.packages("rpart")
library(rpart)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [4]:
options(repr.plot.width=12, repr.plot.height=12)

test_tree_accuracy <- function(df, plot=FALSE) {
 
    # df = select(df, -Color)
    # split into test and training sets
    nrows <- nrow(df)
    smp_size <- floor(0.75 * nrows)

    set.seed(42)
    train_ind <- sample(seq_len(nrows), size = smp_size)
    
    train <- df[train_ind, ]
    test <- df[-train_ind, ]
    
    formula <- as.formula('Class ~ .')

    # grow tree 
    tree_model <- rpart(formula,
                 method="class", data=train, control=rpart.control(minsplit=30, cp=0.01))
    
    if (plot) {
        # plot tree 
        plot(tree_model, uniform=TRUE, 
             main="Classification Tree for Adult Income")
        text(tree_model, use.n=TRUE, all=TRUE, cex=.8)
    }
    
    # Make a prediction & test accuracy
    test_pred <- predict(tree_model, test, type = "class")
    
    ### junky debug stuff
    # print(head (test_pred))
    ### junky debug stuff
    accuracy <- sum(test[,"Class"]==test_pred)/length(test_pred)
    print("-----------------------< Features Used >----------------------------------")
    print(colnames(df))
    print(paste("Accuracy=", round(accuracy*100,2), "%", sep=""))
    print("------------------------------------")
    print(paste(''))
}

# Accuracy With All Attributes

In [5]:
test_tree_accuracy(data)

[1] "-----------------------< Features Used >----------------------------------"
 [1] "fixed.acidity"        "volatile.acidity"     "citric.acid"         
 [4] "residual.sugar"       "chlorides"            "free.sulfur.dioxide" 
 [7] "total.sulfur.dioxide" "density"              "pH"                  
[10] "sulphates"            "alcohol"              "quality"             
[13] "Class"               
[1] "Accuracy=97.72%"
[1] "------------------------------------"
[1] ""


Okay, 97.72% accuracy (NBC was 97.91%). Not bad.  Let's start keeping score.

| Configuration | Tree Accuracy | NBC Acuracy |
|---------------|---------------|-------------|
|All features   | 97.72%        |  97.91%     |

# Accuracy With Minimal Subset and 95% Threshold

## Simplest Case First: Single-Feature Sensor
Presumably, the cheapest sensor we can design and build is one that samples only a single paremeter. Given our small search space (6 features) and reusable function, we can do a quick check in six lines of code

In [6]:
colnames(data)

[1] "fixed.acidity"        "volatile.acidity"     "citric.acid"         
 [4] "residual.sugar"       "chlorides"            "free.sulfur.dioxide" 
 [7] "total.sulfur.dioxide" "density"              "pH"                  
[10] "sulphates"            "alcohol"              "quality"             
[13] "Class"

In [7]:
test_tree_accuracy(select(data, fixed.acidity, Class))
test_tree_accuracy(select(data, volatile.acidity, Class))
test_tree_accuracy(select(data, citric.acid, Class))
test_tree_accuracy(select(data, residual.sugar, Class))
test_tree_accuracy(select(data, chlorides, Class))
test_tree_accuracy(select(data, free.sulfur.dioxide, Class))
test_tree_accuracy(select(data, total.sulfur.dioxide, Class))
test_tree_accuracy(select(data, density, Class))
test_tree_accuracy(select(data, pH, Class))
test_tree_accuracy(select(data, sulphates, Class))
test_tree_accuracy(select(data, alcohol, Class))

[1] "-----------------------< Features Used >----------------------------------"
[1] "fixed.acidity" "Class"        
[1] "Accuracy=80.92%"
[1] "------------------------------------"
[1] ""
[1] "-----------------------< Features Used >----------------------------------"
[1] "volatile.acidity" "Class"           
[1] "Accuracy=86.22%"
[1] "------------------------------------"
[1] ""
[1] "-----------------------< Features Used >----------------------------------"
[1] "citric.acid" "Class"      
[1] "Accuracy=80.25%"
[1] "------------------------------------"
[1] ""
[1] "-----------------------< Features Used >----------------------------------"
[1] "residual.sugar" "Class"         
[1] "Accuracy=83.57%"
[1] "------------------------------------"
[1] ""
[1] "-----------------------< Features Used >----------------------------------"
[1] "chlorides" "Class"    
[1] "Accuracy=91.94%"
[1] "------------------------------------"
[1] ""
[1] "-----------------------< Features Used >--------------

None of the single-feature tests reached our 95% thrshold, butit's worth noting two of them passed 90%.  We'll record those for fun, comparing to their performance under NBC.


| Configuration                 | Tree Accuracy | NBC Acuracy |
|-------------------------------|---------------|-------------|
|All features                   | 97.72%        | 97.91%      |
|1 feature (sulfur.dioxide)     | 93.05%        | 93.11%      |
|1 feature (chloride)           | 91.94%        | 79.57%      |

Trying a few tricks recommended by the [Tree-Based Models article](https://www.statmethods.net/advstats/cart.html) from [Quick-R](https://www.statmethods.net/index.html)

In [8]:
fit <- rpart (Class ~ ., method="class", data=data)
printcp (fit)


Classification tree:
rpart(formula = Class ~ ., data = data, method = "class")

Variables actually used in tree construction:
[1] chlorides            total.sulfur.dioxide volatile.acidity    

Root node error: 1599/6497 = 0.24611

n= 6497 

        CP nsplit rel error   xerror      xstd
1 0.700438      0  1.000000 1.000000 0.0217134
2 0.067542      1  0.299562 0.332083 0.0138097
3 0.062539      2  0.232020 0.255159 0.0122292
4 0.017511      4  0.106942 0.116948 0.0084281
5 0.016260      5  0.089431 0.101939 0.0078837
6 0.010000      6  0.073171 0.090056 0.0074211


In [9]:
options(repr.plot.width=8, repr.plot.height=4)
plotcp(fit)

In [10]:
summary(fit)

Call:
rpart(formula = Class ~ ., data = data, method = "class")
  n= 6497 

          CP nsplit  rel error     xerror        xstd
1 0.70043777      0 1.00000000 1.00000000 0.021713445
2 0.06754221      1 0.29956223 0.33208255 0.013809692
3 0.06253909      2 0.23202001 0.25515947 0.012229203
4 0.01751094      4 0.10694184 0.11694809 0.008428118
5 0.01626016      5 0.08943089 0.10193871 0.007883661
6 0.01000000      6 0.07317073 0.09005629 0.007421057

Variable importance
total.sulfur.dioxide            chlorides  free.sulfur.dioxide 
                  34                   21                   13 
    volatile.acidity        fixed.acidity          citric.acid 
                  11                    7                    6 
             density            sulphates       residual.sugar 
                   3                    2                    2 
                  pH 
                   1 

Node number 1: 6497 observations,    complexity param=0.7004378
  predicted class=0  expected lo

Okay, the summary says Total Sulfur Dioxide is the most important, follwed by Choride. The plot(cp) sugests we can get decent performance improvements for up to 4 variables, but after that the law of diminishing returns kicks in.

We'll use our handy test_tree_accuracy() function start with the single most important feature, then add one more each time until we get our Top Four.

In [11]:
test_tree_accuracy(select(data, one_of(c('Class','total.sulfur.dioxide'))))
test_tree_accuracy(select(data, one_of(c('Class','total.sulfur.dioxide', 'chlorides'))))
test_tree_accuracy(select(data, one_of(c('Class','total.sulfur.dioxide', 'chlorides', 
                                         'free.sulfur.dioxide'))))
test_tree_accuracy(select(data, one_of(c('Class','total.sulfur.dioxide', 'chlorides', 
                                         'free.sulfur.dioxide', 'volatile.acidity'))))

[1] "-----------------------< Features Used >----------------------------------"
[1] "Class"                "total.sulfur.dioxide"
[1] "Accuracy=93.05%"
[1] "------------------------------------"
[1] ""
[1] "-----------------------< Features Used >----------------------------------"
[1] "Class"                "total.sulfur.dioxide" "chlorides"           
[1] "Accuracy=96.86%"
[1] "------------------------------------"
[1] ""
[1] "-----------------------< Features Used >----------------------------------"
[1] "Class"                "total.sulfur.dioxide" "chlorides"           
[4] "free.sulfur.dioxide" 
[1] "Accuracy=96.86%"
[1] "------------------------------------"
[1] ""
[1] "-----------------------< Features Used >----------------------------------"
[1] "Class"                "total.sulfur.dioxide" "chlorides"           
[4] "free.sulfur.dioxide"  "volatile.acidity"    
[1] "Accuracy=97.29%"
[1] "------------------------------------"
[1] ""


**We can hit our thrshold with two features -- Sulfur Dioxide and Chorides**

| Configuration                        | Tree Accuracy | NBC Acuracy |
|--------------------------------------|---------------|-------------|
|All features                          | 97.72%        | 97.91%      |
|1 feature (sulfur.dioxide)            | 93.05%        | 93.11%      |
|1 feature (chloride)                  | 91.94%        | 79.57%      |
|2 features (sulfur.dioxide, chloride) | 96.86% | n/a |

# Visualizaing the Decision Tree

In [12]:
options(repr.plot.width=8, repr.plot.height=9)

## Start with the full tree in all its glory

In [13]:
test_tree_accuracy(data, plot=TRUE)

[1] "-----------------------< Features Used >----------------------------------"
 [1] "fixed.acidity"        "volatile.acidity"     "citric.acid"         
 [4] "residual.sugar"       "chlorides"            "free.sulfur.dioxide" 
 [7] "total.sulfur.dioxide" "density"              "pH"                  
[10] "sulphates"            "alcohol"              "quality"             
[13] "Class"               
[1] "Accuracy=97.72%"
[1] "------------------------------------"
[1] ""


Got some pretty small observation counts at the leaves ... even zero for total.sulfur.dioxide>=156

## Next the two-feature tree that's just-enough to get us past our 95% threshold

In [14]:
test_tree_accuracy(select(data, one_of(c('Class','total.sulfur.dioxide', 'chlorides'))), plot=TRUE)

[1] "-----------------------< Features Used >----------------------------------"
[1] "Class"                "total.sulfur.dioxide" "chlorides"           
[1] "Accuracy=96.86%"
[1] "------------------------------------"
[1] ""


Most of the leaves are more robust, but the very lowest stil has 6 and 9.

# Finding the Minimum Number of Attributes

In the analysis above, we tried all single-attribute tests and never got better than 93% accuracy. With a bit of CART modeling, we pruned down to two.


# Comparing to Naive Bayesian Classifier

Taking the leaderboard from  HW1 and comparing to what we have so far ...

|Features| Configuration                                   |NBC <br> Accuracy| Tree <br> Accuracy |
|--------|:------------------------------------------------|:------:|:------:|
| 12     |All features                                     | 97.91% | 97.72% |
| 1      |sulfur.dioxide                                   | 93.11% | 93.05% |
| 10     |All but pH and citric.acid                       | 97.41% |   n/a  |
| 4      |total.sulfur.dioxide, sulphates, alcohol, quality| 95.02% |   n/a  |
| 3      |total.sulfur.dioxide, sulphates, alcohol         | 94.89% |   n/a  |
| 2      |total.sulfur.dioxide, sulphates                  | 94.71% |   n/a  |
| 2      |total.sulfur.dioxide, chloride                   |   n/a  | 96.86% |

Running a few more tests to flesh out the leaderboard .., 

In [15]:
test_tree_accuracy(select(data, -one_of(c('pH','citric.acid'))))
test_tree_accuracy(select(data, one_of(c('total.sulfur.dioxide', 'sulphates', 'alcohol', 'quality', 'Class'))))
test_tree_accuracy(select(data, one_of(c('total.sulfur.dioxide', 'sulphates', 'alcohol', 'Class' ))))
test_tree_accuracy(select(data, one_of(c('total.sulfur.dioxide', 'sulphates', 'Class' ))))


[1] "-----------------------< Features Used >----------------------------------"
 [1] "fixed.acidity"        "volatile.acidity"     "residual.sugar"      
 [4] "chlorides"            "free.sulfur.dioxide"  "total.sulfur.dioxide"
 [7] "density"              "sulphates"            "alcohol"             
[10] "quality"              "Class"               
[1] "Accuracy=97.72%"
[1] "------------------------------------"
[1] ""
[1] "-----------------------< Features Used >----------------------------------"
[1] "total.sulfur.dioxide" "sulphates"            "alcohol"             
[4] "quality"              "Class"               
[1] "Accuracy=95.26%"
[1] "------------------------------------"
[1] ""
[1] "-----------------------< Features Used >----------------------------------"
[1] "total.sulfur.dioxide" "sulphates"            "alcohol"             
[4] "Class"               
[1] "Accuracy=95.26%"
[1] "------------------------------------"
[1] ""
[1] "-----------------------< Features Used >

|Features| Configuration                                    |NBC <br> Accuracy| Tree <br> Accuracy |
|--------|:-------------------------------------------------|:------:|:------:|
| 12     |All features                                      | 97.91% | 97.72% |
| 1      |sulfur.dioxide                                    | 93.11% | 93.05% |
| 10     |All but pH and citric.acid                        | 97.41% | 97.72% |
| 4      |total.sulfur.dioxide, sulphates, alcohol, quality | 95.02% | 95.26% |
| 3      |total.sulfur.dioxide, sulphates, alcohol          | 94.89% | 95.26% |
| 2      |total.sulfur.dioxide, sulphates                   | 94.71% | 93.60% |
| 2      |total.sulfur.dioxide, chloride                    | 93.11% | 96.86% |

Performance of the two models is very similar, but it's interesting to see how they "flirt" with a 95% threshold. If you allow for rounding to a "whole" percentage, NBC predicts your best two-sensor model would detect total sulfur dioxide + sulphates, while the Decision Tree predicts total sulfur dioxid + chloride.

# ==============< JUNK  V2 >===================================

## Another try at pruning

In [40]:
 # grow tree 
fit <- rpart(Class ~ .,
    method="class", data=data)

In [41]:
pfit<- prune(fit, cp=   fit$cptable[which.min(fit$cptable[,"xerror"]),"CP"])

Update the leaderboard

|Features| Configuration  | Tree Accuracy | NBC Acuracy |
|--------|----------------|---------------|-------------|
| 12     |All features    | 97.72%        | 97.91%      |
| 1      |sulfur.dioxide  | 93.05%        | 93.11%      |
| 1      |chloride        | 91.94%        | 79.57%      |
| 4      |total.sulfur.dioxide, chlorides, <br> free.sulfure.dioxide, volatile.acidity | 97.29% | n/a |


# Try the ID3 Approach

In [16]:
install.packages("data.tree", repos=c("http://rstudio.org/_packages", "http://cran.rstudio.com"))

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [17]:
library(data.tree)

In [18]:
data_id3 = data
data_id3$Color <- ifelse(data_id3$Class ==1,'Red','White')
data_id3$Color <- factor(data_id3$Color)
head(data_id3)

fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality,Class,Color
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,1,Red
7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5,1,Red
7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5,1,Red
11.2,0.28,0.56,1.9,0.075,17,60,0.9980,3.16,0.58,9.8,6,1,Red
7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,1,Red
7.4,0.66,0.00,1.8,0.075,13,40,0.9978,3.51,0.56,9.4,5,1,Red


In [19]:
data_id3 = select(data_id3, -Class)
data_id3 = 
head(data_id3)

In [20]:
# Compute purity
IsPure <- function(data) {
  length(unique(data[,ncol(data)])) == 1
}

In [21]:
# Compute Entropy - purity of a dataset
Entropy <- function( vls ) {
  res <- vls/sum(vls) * log2(vls/sum(vls))
  res[vls == 0] <- 0
  -sum(res)
}

In [22]:
# Information Gain
#  Measures the difference between the entropy before the split, and the weighted sum of the entropies after the split.
InformationGain <- function( tble ) {
  entropyBefore <- Entropy(colSums(tble))
  s <- rowSums(tble)
  entropyAfter <- sum (s / sum(s) * apply(tble, MARGIN = 1, FUN = Entropy ))
  informationGain <- entropyBefore - entropyAfter
  return (informationGain)
}

In [23]:
#Training
TrainID3 <- function(node, data) {
    
  node$obsCount <- nrow(data)
  
  #if the data-set is pure (e.g. all toxic), then
  if (IsPure(data)) {
    #construct a leaf having the name of the pure feature (e.g. 'toxic')
    child <- node$AddChild(unique(data[,ncol(data)]))
    node$feature <- tail(names(data), 1)
    child$obsCount <- nrow(data)
    child$feature <- ''
  } else {
    #calculate the information gain
    ig <- sapply(colnames(data)[-ncol(data)], 
            function(x) InformationGain(
              table(data[,x], data[,ncol(data)])
              )
            )
    #chose the feature with the highest information gain (e.g. 'color')
    #if more than one feature have the same information gain, then take
    #the first one
    feature <- names(which.max(ig))
    node$feature <- feature
    
    #take the subset of the data-set having that feature value
    
    childObs <- split(data[ ,names(data) != feature, drop = FALSE], 
                      data[ ,feature], 
                      drop = TRUE)
    if (length(childObs) > 0) {
         for(i in 1:length(childObs)) {
          #construct a child having the name of that feature value (e.g. 'red')
          child <- node$AddChild(names(childObs)[i])

          #call the algorithm recursively on the child and the subset      
          TrainID3(child, childObs[[i]])
        }
    }
    
  }
}

In [24]:
tree <- Node$new("data_id3")
tree

  levelName
1  data_id3

In [25]:
TrainID3(tree, data_id3)
tree
# print(tree, "feature", "obsCount")

  levelName
1  data_id3
2   °--Red 

In [26]:
colnames(data)

[1] "fixed.acidity"        "volatile.acidity"     "citric.acid"         
 [4] "residual.sugar"       "chlorides"            "free.sulfur.dioxide" 
 [7] "total.sulfur.dioxide" "density"              "pH"                  
[10] "sulphates"            "alcohol"              "quality"             
[13] "Class"

In [27]:
tree <- Node$new("data_id3")
TrainID3(tree, select(data, fixed.acidity, Class))
tree

ERROR: Error in .subset2(x, i, exact = exact): attempt to select less than one element in integerOneIndex


In [31]:
data_v1 = select(data, fixed.acidity, Class)
tree <- Node$new("data_v1")
TrainID3(tree, data_v1)
tree

ERROR: Error in .subset2(x, i, exact = exact): attempt to select less than one element in integerOneIndex


In [32]:

tree <- Node$new("data_id3")
TrainID3(tree, select(data, fixed.acidity, volatile.acidity, Class))
tree

ERROR: Error in .subset2(x, i, exact = exact): attempt to select less than one element in integerOneIndex


In [33]:
tree <- Node$new("data_id3")
TrainID3(tree, select(data, fixed.acidity, volatile.acidity, citric.acid, Class))
tree

ERROR: Error in .subset2(x, i, exact = exact): attempt to select less than one element in integerOneIndex


In [34]:
tree <- Node$new("data_id3")
TrainID3(tree, select(data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, Class))
tree

ERROR: Error in .subset2(x, i, exact = exact): attempt to select less than one element in integerOneIndex


In [ ]:
tree <- Node$new("data_id3")
TrainID3(tree, select(data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, chlorides, Class))
tree

In [39]:
tree <- Node$new("data_id3")
TrainID3(tree, select(data, fixed.acidity, volatile.acidity, chlorides, Class))
tree

ERROR: Error in .subset2(x, i, exact = exact): attempt to select less than one element in integerOneIndex


In [ ]:
tree <- Node$new("data_id3")
TrainID3(tree, select(data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, chlorides,
                      free.sulfur.dioxide, Class))
tree

In [35]:
tree <- Node$new("data_id3")
TrainID3(tree, select(data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, chlorides,
                      free.sulfur.dioxide, total.sulfur.dioxide, Class))
tree

ERROR: Error in .subset2(x, i, exact = exact): attempt to select less than one element in integerOneIndex


In [ ]:
tree <- Node$new("data_id3")
TrainID3(tree, select(data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, chlorides,
                      free.sulfur.dioxide, total.sulfur.dioxide, density, Class))
tree

In [ ]:
tree <- Node$new("data_id3")
TrainID3(tree, select(data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, chlorides,
                      free.sulfur.dioxide, total.sulfur.dioxide, density, pH, Class))
tree

In [ ]:
tree <- Node$new("data_id3")
TrainID3(tree, select(data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, chlorides,
                      free.sulfur.dioxide, total.sulfur.dioxide, density, pH, sulphates, Class))
tree

In [ ]:
tree <- Node$new("data_id3")
TrainID3(tree, select(data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, chlorides,
                      free.sulfur.dioxide, total.sulfur.dioxide, density, pH, sulphates, alcohol, Class))
tree

In [36]:
tree <- Node$new("data_id3")
TrainID3(tree, select(data, fixed.acidity, volatile.acidity, citric.acid, residual.sugar, chlorides,
                      free.sulfur.dioxide, total.sulfur.dioxide, density, pH, sulphates, alcohol, quality, Class))
tree

ERROR: Error in .subset2(x, i, exact = exact): attempt to select less than one element in integerOneIndex
